# Project

In [ ]:
# Import required libraries
import datetime
import time
import polygon
from sqlalchemy import create_engine 
from sqlalchemy import text
import pandas as pd
from math import sqrt
from math import isnan
import matplotlib.pyplot as plt
from numpy import mean
from numpy import std
from math import floor
import numpy as np
import sys

In [ ]:
import json
import requests

In [ ]:
import sqlite3

In [ ]:
#fred API key -> ea55b200b69da85cd36a010a0b180022
#pip install fredapi
from fredapi import Fred
fred = Fred(api_key='ea55b200b69da85cd36a010a0b180022')

In [ ]:
# import data
dfii5 = fred.get_series('DFII5')
dgs5 = fred.get_series('DGS5')
t5yie = fred.get_series('T5YIE')

dfii5.name = 'DFII5'
dgs5.name = 'DGS5'
t5yie.name = 'T5YIE'

#fred.search('DFII5').T
#DFII5,DGS5,T5YIE to get description of series

In [ ]:
dfii5

In [ ]:
df1 = pd.merge(dfii5, dgs5, left_index=True, right_index=True)
df2 = pd.merge(df1, t5yie, left_index=True, right_index=True)
df2

In [ ]:
#df2.index.get_loc('2015-01-30')
#df2['DGS5'][12]
df2.isnull().sum()

In [ ]:
# Replace missing values by avg of previous and next values

for i in range (0,len(df2)):
    if np.isnan(df2['DFII5'][i]):
        df2['DFII5'][i] = (df2['DFII5'][i-1] + df2['DFII5'][i+1])/2

for i in range (0,len(df2)):
    if np.isnan(df2['DGS5'][i]):
        df2['DGS5'][i] = (df2['DGS5'][i-1] + df2['DGS5'][i+1])/2 
            
            
for i in range (0,len(df2)):
    if np.isnan(df2['T5YIE'][i]):
        df2['T5YIE'][i] = (df2['T5YIE'][i-1] + df2['T5YIE'][i+1])/2 
            


In [ ]:
#Resort and calculate coefficients (daily changes): 1 + (Pf – Pi) / Pf
df2["Coefficient"] = " "

for i in range (1,len(df2)):
    df2['Coefficient'][i] = 1 + ( ( df2['T5YIE'][i] - df2['T5YIE'][i-1] ) / df2['T5YIE'][i] )
       

In [ ]:
df2

In [ ]:
#BLS API key = Your API key is: f32f516d65664a19b4d1583b6ec7abd2

#CUUR0000SA0 : All items in U.S. city average, all urban consumers, not seasonally adjusted
#CUUR0000SA0L1E : All items less food and energy in U.S. city average, all urban consumers, not seasonally adjusted

In [ ]:
#import config # .py file with bls_key = 'API key here'

# The url for BLS API v2
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

# API key 
key = '?registrationkey={}'.format('f32f516d65664a19b4d1583b6ec7abd2')

# Series stored as a dictionary
series_dict = {
    'CUUR0000SA0': 'All items',
    'CUUR0000SA0L1E': 'All items less food and energy'}


In [ ]:
# Start year and end year
dates = ('2002', '2012')

# Specify json as content type to return
headers = {'Content-type': 'application/json'}

# Submit the list of series as data
data = json.dumps({
    "seriesid": list(series_dict.keys()),
    "startyear": dates[0],
    "endyear": dates[1]})

# Post request for the data
p = requests.post(
    '{}{}'.format(url, key),
    headers=headers,
    data=data).json()['Results']['series']

In [ ]:
# Date index from first series
date_list = [f"{i['year']}-{i['period'][1:]}-01" for i in p[0]['data']]

# Empty dataframe to fill with values
df_bls1 = pd.DataFrame()

# Build a pandas series from the API results, p
for s in p:
    df_bls1[series_dict[s['seriesID']]] = pd.Series(
        index = pd.to_datetime(date_list),
        data = [i['value'] for i in s['data']]
        ).astype(float).iloc[::-1]


In [ ]:
df_bls1

In [ ]:
# For the next 10 years data
# Start year and end year
dates2 = ('2012', '2020')

# Specify json as content type to return
headers = {'Content-type': 'application/json'}

# Submit the list of series as data
data2 = json.dumps({
    "seriesid": list(series_dict.keys()),
    "startyear": dates2[0],
    "endyear": dates2[1]})

# Post request for the data
p = requests.post(
    '{}{}'.format(url, key),
    headers=headers,
    data=data2).json()['Results']['series']

# Date index from first series
date_list = [f"{i['year']}-{i['period'][1:]}-01" for i in p[0]['data']]

# Empty dataframe to fill with values
df_bls2 = pd.DataFrame()

# Build a pandas series from the API results, p
for s in p:
    df_bls2[series_dict[s['seriesID']]] = pd.Series(
        index = pd.to_datetime(date_list),
        data = [i['value'] for i in s['data']]
        ).astype(float).iloc[::-1]

In [ ]:
df_bls2

In [ ]:
# For the next 10 years data
# Start year and end year
dates3 = ('2021', '2022')

# Specify json as content type to return
headers = {'Content-type': 'application/json'}

# Submit the list of series as data
data3 = json.dumps({
    "seriesid": list(series_dict.keys()),
    "startyear": dates3[0],
    "endyear": dates3[1]})

# Post request for the data
p = requests.post(
    '{}{}'.format(url, key),
    headers=headers,
    data=data3).json()['Results']['series']

# Date index from first series
date_list = [f"{i['year']}-{i['period'][1:]}-01" for i in p[0]['data']]

# Empty dataframe to fill with values
df_bls3 = pd.DataFrame()

# Build a pandas series from the API results, p
for s in p:
    df_bls3[series_dict[s['seriesID']]] = pd.Series(
        index = pd.to_datetime(date_list),
        data = [i['value'] for i in s['data']]
        ).astype(float).iloc[::-1]

In [ ]:
df_bls3

In [ ]:
df_bls4 = df_bls1.append(df_bls2)
df_bls = df_bls4.append(df_bls3)
df_bls

In [ ]:
#calculate 12 month percent change to get % inflation data
df_bls["Inflation"] = " "
df_bls["Elastic inflation"] = " "
df_bls["Inelastic inflation"] = " "

for i in range (12,len(df_bls)):
    df_bls['Inflation'][i] = ( df_bls['All items'][i] - df_bls['All items'][i-12] ) / df_bls['All items'][i-12] 
    df_bls['Elastic inflation'][i] = ( df_bls['All items less food and energy'][i] - df_bls['All items less food and energy'][i-12] ) / df_bls['All items less food and energy'][i-12] 
    df_bls['Inelastic inflation'][i] = df_bls['Inflation'][i] + df_bls['Inflation'][i] - df_bls['Elastic inflation'][i]
    
    
    

In [ ]:
df_bls

In [ ]:
df2["Date"] = " "

for i in range (0,len(df2)):
    df2['Date'][i] = df2.index[i]

In [ ]:
#Merge the two dataframes 
#Input the monthly values to each day's values

final =pd.merge(
    df_bls[["Inflation", "Elastic inflation", "Inelastic inflation"]],
    df2,
    left_on=[df_bls.index.year, df_bls.index.month],
    right_on=[df2.index.year, df2.index.month],
    how="right",
    suffixes=("bio", "tc_"),
).drop(["key_0", "key_1"], axis=1)


#    left_on=[bio["Date"].dt.year, bio["Date"].dt.month],
#    right_on=[tc["Date"].dt.year, tc["Date"].dt.month],


In [ ]:
final

In [ ]:
#prev day inflation * today's coefficient gives daily values
final["Inflation_daily"] = " "
final["Elastic_inflation_daily"] = " "
final["Inelastic_inflation_daily"] = " "

for i in range (1,len(final)):
    final["Inflation_daily"][i] = final["Inflation"][i-1] * final["Coefficient"][i]
    final["Elastic_inflation_daily"][i] = final["Elastic inflation"][i-1] * final["Coefficient"][i]
    final["Inelastic_inflation_daily"][i] = final["Inelastic inflation"][i-1] * final["Coefficient"][i]

In [ ]:
#  daily smooth values 
# if abs(current daily - prev daily)<0.0025 then current daily else avg(current, prev daily)
final["Inflation_daily_smooth"] = " "
final["Elastic_inflation_daily_smooth"] = " "
final["Inelastic_inflation_daily_smooth"] = " "

for i in range (2,len(final)):
    if abs(final["Inflation_daily"][i] - final["Inflation_daily"][i-1]) < 0.0025:
        final["Inflation_daily_smooth"][i] = final["Inflation_daily"][i]
    else:
        final["Inflation_daily_smooth"][i] = (final["Inflation_daily"][i] + final["Inflation_daily"][i-1]) / 2
 

In [ ]:
for i in range (2,len(final)):
    if abs(final["Elastic_inflation_daily"][i] - final["Elastic_inflation_daily"][i-1]) < 0.0025:
        final["Elastic_inflation_daily_smooth"][i] = final["Elastic_inflation_daily"][i]
    else:
        final["Elastic_inflation_daily_smooth"][i] = (final["Elastic_inflation_daily"][i] + final["Elastic_inflation_daily"][i-1]) / 2



In [ ]:
for i in range (2,len(final)):
    if abs(final["Inelastic_inflation_daily"][i] - final["Inelastic_inflation_daily"][i-1]) < 0.0025:
        final["Inelastic_inflation_daily_smooth"][i] = final["Inelastic_inflation_daily"][i]
    else:
        final["Inelastic_inflation_daily_smooth"][i] = (final["Inelastic_inflation_daily"][i] + final["Inelastic_inflation_daily"][i-1]) / 2


In [ ]:
final = final.drop(['Inflation','Elastic inflation','Inelastic inflation','DFII5','DGS5','T5YIE','Coefficient','Inflation_daily','Elastic_inflation_daily','Inelastic_inflation_daily'], axis=1)

final.drop(final.index[[0,1]], inplace=True)

final = final.dropna(how='any')


In [ ]:
final['Date'] = pd.to_datetime(final['Date']).dt.date
final

In [ ]:
# plot two lines
plt.plot(final["Date"], final["Inflation_daily_smooth"], 'o-r')
plt.plot(final["Date"], final["Elastic_inflation_daily_smooth"], 'o-b')
plt.plot(final["Date"], final["Inelastic_inflation_daily_smooth"], 'o-g')
# set axis titles
plt.xlabel("Date")
plt.ylabel("Inflation")
# set chart title
plt.title("Inflation levels")
# legend
plt.legend(['Full', 'elastic','inelastic'])
plt.show()

In [ ]:
#!pip install yfinance

In [ ]:
import yfinance as yf  

In [ ]:
currencyrates_yf = yf.download([ 'EURUSD=X', 'USDINR=X','GBPUSD=X' ],'2003-01-06','2022-07-01')


In [ ]:
currencyrates_yf_adjclose = currencyrates_yf["Adj Close"]

In [ ]:
currencyrates_yf_adjclose

In [ ]:
currencyrates_yf_adjclose["Date"] = " "

for i in range (0,len(currencyrates_yf_adjclose)):
    currencyrates_yf_adjclose['Date'][i] = currencyrates_yf_adjclose.index[i]

In [ ]:
currencyrates_yf_adjclose['Date'] = pd.to_datetime(currencyrates_yf_adjclose['Date']).dt.date

In [ ]:
table5 =pd.merge(
    final,
    currencyrates_yf_adjclose,
    right_on=[currencyrates_yf_adjclose['Date']],
    left_on=[final['Date']],
    how="inner"
).drop(["key_0","Date_y"], axis=1)

In [ ]:
table5.rename(columns = {'Date_x':'Date','Inflation_daily_smooth':'Full_inflation',
                         'Elastic_inflation_daily_smooth':'Elastic_inflation',
                         'Inelastic_inflation_daily_smooth':'Inelastic_inflation',
                         'EURUSD=X':'EURUSD','GBPUSD=X':'GBPUSD','USDINR=X':'USDINR'}, inplace = True)

In [ ]:
#Sort data in desc. order
table5 = table5.sort_values(by='Date',ascending=False)

In [ ]:
#Clean nulls
for i in range (0,len(table5)):
    if np.isnan(table5['EURUSD'][i]):
        table5['EURUSD'][i] = (table5['EURUSD'][i-1] + table5['EURUSD'][i+1])/2
        if np.isnan(table5['EURUSD'][i]):
            table5['EURUSD'][i] = table5['EURUSD'].mean()
            
        
for i in range (0,len(table5)):
    if np.isnan(table5['GBPUSD'][i]):
        table5['GBPUSD'][i] = (table5['GBPUSD'][i-1] + table5['GBPUSD'][i+1])/2
        if np.isnan(table5['GBPUSD'][i]):
            table5['GBPUSD'][i] = table5['GBPUSD'].mean()
        
for i in range (0,len(table5)):
    if np.isnan(table5['USDINR'][i]):
        table5['USDINR'][i] = (table5['USDINR'][i-1] + table5['USDINR'][i+1])/2
        if np.isnan(table5['USDINR'][i]):
            table5['USDINR'][i] = table5['USDINR'].mean()

In [ ]:
table6 = table5

In [ ]:
# Correcting the index 
table6 = table6.reset_index()
table6 = table6.drop(['index'],axis=1)

In [ ]:
table6

In [ ]:
table6['Week_tag']=""
i=0
while i < len(table6):
    table6['Week_tag'][i] = 1
    i = i + 5
    

In [ ]:
#Sort data in asc. order for rolling correlations
table6 = table6.sort_values(by='Date',ascending=True)

In [ ]:
table6['EURUSD_fi'] = ""
table6['EURUSD_ei'] = ""
table6['EURUSD_ii'] = ""

table6['EURUSD_fi'] = table6['Full_inflation'].rolling(100).corr(table6['EURUSD'])
table6['EURUSD_ei'] = table6['Elastic_inflation'].rolling(100).corr(table6['EURUSD'])
table6['EURUSD_ii'] = table6['Inelastic_inflation'].rolling(100).corr(table6['EURUSD'])

In [ ]:
table6['GBPUSD_fi'] = ""
table6['GBPUSD_ei'] = ""
table6['GBPUSD_ii'] = ""

table6['GBPUSD_fi'] = table6['Full_inflation'].rolling(100).corr(table6['GBPUSD'])
table6['GBPUSD_ei'] = table6['Elastic_inflation'].rolling(100).corr(table6['GBPUSD'])
table6['GBPUSD_ii'] = table6['Inelastic_inflation'].rolling(100).corr(table6['GBPUSD'])

In [ ]:
table6['USDINR_fi'] = ""
table6['USDINR_ei'] = ""
table6['USDINR_ii'] = ""

table6['USDINR_fi'] = table6['Full_inflation'].rolling(100).corr(table6['USDINR'])
table6['USDINR_ei'] = table6['Elastic_inflation'].rolling(100).corr(table6['USDINR'])
table6['USDINR_ii'] = table6['Inelastic_inflation'].rolling(100).corr(table6['USDINR'])

In [ ]:
table8 = table6.dropna()

In [ ]:
#Sort data in desc. order 
table8 = table8.sort_values(by='Date',ascending=False)

In [297]:
table8

,Date,Full_inflation,Elastic_inflation,Inelastic_inflation,EURUSD,GBPUSD,USDINR,Week_tag,EURUSD_fi,EURUSD_ei,EURUSD_ii,GBPUSD_fi,GBPUSD_ei,GBPUSD_ii,USDINR_fi,USDINR_ei,USDINR_ii
0,2022-07-01,0.091294,0.059625,0.122964,1.047768,1.216086,78.962997,1,-0.599368,0.799134,-0.756041,-0.653402,0.848099,-0.818215,0.775698,-0.754709,0.885303
1,2022-06-30,0.087973,0.057456,0.118491,1.044659,1.213180,78.898300,,-0.605610,0.796368,-0.760150,-0.654162,0.848870,-0.818855,0.770723,-0.760169,0.882404
2,2022-06-29,0.087153,0.055971,0.117387,1.052355,1.218977,79.073196,,-0.613220,0.793003,-0.763366,-0.659341,0.846814,-0.820355,0.779104,-0.755801,0.886414
3,2022-06-28,0.088606,0.057869,0.119344,1.058089,1.226738,78.441101,,-0.623391,0.792137,-0.767799,-0.667021,0.844822,-0.822466,0.794379,-0.743341,0.893932
4,2022-06-27,0.092199,0.060215,0.124182,1.056613,1.228245,78.252296,,-0.630512,0.790798,-0.772268,-0.670115,0.843138,-0.823391,0.798061,-0.738117,0.894907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4734,2004-04-22,0.021979,0.016936,0.027023,1.192506,1.775285,43.938999,,-0.549151,-0.748592,-0.072041,-0.348609,0.111119,-0.436621,-0.602574,-0.822269,-0.081637
4735,2004-04-21,0.023504,0.018111,0.028897,1.181000,1.769504,43.981998,1,-0.530404,-0.722895,-0.064713,-0.328042,0.144663,-0.432882,-0.588932,-0.819925,-0.063575
4736,2004-04-20,0.022083,0.017016,0.02715,1.184806,1.785300,43.868000,,-0.497198,-0.693603,-0.040240,-0.303760,0.185496,-0.426136,-0.572254,-0.816366,-0.037604
4737,2004-04-19,0.023037,0.017751,0.028323,1.200696,1.807109,43.790001,,-0.471939,-0.665744,-0.028517,-0.290509,0.212246,-0.425188,-0.553977,-0.813215,-0.015879


In [ ]:
dat = sqlite3.connect("./sqlite/final008.db")
query = dat.execute("SELECT * From USDINR_agg_kerneldata")
cols = [column[0] for column in query.description]
results_usdinr = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)

In [ ]:
results_eurusd

In [ ]:
dat = sqlite3.connect("./sqlite/final008.db")
query = dat.execute("SELECT * From EURUSD_agg_kerneldata")
cols = [column[0] for column in query.description]
results_eurusd = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)

In [ ]:
query = dat.execute("SELECT * From GBPUSD_agg_kerneldata")
cols = [column[0] for column in query.description]
results_gbpusd = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)

In [ ]:
EURUSD = table8.drop(['Full_inflation','Elastic_inflation','Inelastic_inflation',
                      'EURUSD','GBPUSD','USDINR',
                      'GBPUSD_fi','GBPUSD_ei','GBPUSD_ii',
                      'USDINR_fi','USDINR_ei','USDINR_ii'
                     ],axis=1)


In [ ]:
GBPUSD = table8.drop(['Full_inflation','Elastic_inflation','Inelastic_inflation',
                      'EURUSD','GBPUSD','USDINR',
                      'EURUSD_fi','EURUSD_ei','EURUSD_ii',
                      'USDINR_fi','USDINR_ei','USDINR_ii'
                     ],axis=1)

In [ ]:
USDINR = table8.drop(['Full_inflation','Elastic_inflation','Inelastic_inflation',
                      'EURUSD','GBPUSD','USDINR',
                      'EURUSD_fi','EURUSD_ei','EURUSD_ii',
                      'GBPUSD_fi','GBPUSD_ei','GBPUSD_ii'
                     ],axis=1)

In [ ]:
EURUSD

In [ ]:
GBPUSD

In [ ]:
USDINR

In [ ]:
date_list_calc = list()

date_list_calc.append( USDINR['Date'][0] - datetime.timedelta(30) )

for i in range(1,230):
    date_list_calc.append( date_list_calc[i-1] - datetime.timedelta(30) )
    

In [ ]:
USDINR = USDINR.reset_index()
USDINR = USDINR.drop('index',axis=1)

In [ ]:
EURUSD = EURUSD.reset_index()
EURUSD = EURUSD.drop('index',axis=1)
GBPUSD = GBPUSD.reset_index()
GBPUSD = GBPUSD.drop('index',axis=1)

In [ ]:
USDINR['timestamp'] = ""
USDINR['max'] = ""
USDINR['min'] = ""
USDINR['mean'] = ""
USDINR['vol'] = ""
USDINR['fd'] = ""

j=0

for i in range(0,len(USDINR)):
    if j<len(results_usdinr):
        if USDINR['Date'][i] >= date_list_calc[j]:
            USDINR['timestamp'][i] = results_usdinr['inserttime'][j]
            USDINR['max'][i] = results_usdinr['maxrate'][j]
            USDINR['min'][i] = results_usdinr['minrate'][j]
            USDINR['mean'][i] = results_usdinr['meanrate'][j]
            USDINR['vol'][i] = results_usdinr['vol'][j]
            USDINR['fd'][i] = results_usdinr['fd'][j]
        else:
            USDINR['timestamp'][i] = results_usdinr['inserttime'][j]
            USDINR['max'][i] = results_usdinr['maxrate'][j]
            USDINR['min'][i] = results_usdinr['minrate'][j]
            USDINR['mean'][i] = results_usdinr['meanrate'][j]
            USDINR['vol'][i] = results_usdinr['vol'][j]
            USDINR['fd'][i] = results_usdinr['fd'][j]
            j = j+1
            
    

In [ ]:
EURUSD['timestamp'] = ""
EURUSD['max'] = ""
EURUSD['min'] = ""
EURUSD['mean'] = ""
EURUSD['vol'] = ""
EURUSD['fd'] = ""

j=0

for i in range(0,len(EURUSD)):
    if j<len(results_eurusd):
        if EURUSD['Date'][i] >= date_list_calc[j]:
            EURUSD['timestamp'][i] = results_eurusd['inserttime'][j]
            EURUSD['max'][i] = results_eurusd['maxrate'][j]
            EURUSD['min'][i] = results_eurusd['minrate'][j]
            EURUSD['mean'][i] = results_eurusd['meanrate'][j]
            EURUSD['vol'][i] = results_eurusd['vol'][j]
            EURUSD['fd'][i] = results_eurusd['fd'][j]
        else:
            EURUSD['timestamp'][i] = results_eurusd['inserttime'][j]
            EURUSD['max'][i] = results_eurusd['maxrate'][j]
            EURUSD['min'][i] = results_eurusd['minrate'][j]
            EURUSD['mean'][i] = results_eurusd['meanrate'][j]
            EURUSD['vol'][i] = results_eurusd['vol'][j]
            EURUSD['fd'][i] = results_eurusd['fd'][j]
            j = j+1
            

In [ ]:
GBPUSD['timestamp'] = ""
GBPUSD['max'] = ""
GBPUSD['min'] = ""
GBPUSD['mean'] = ""
GBPUSD['vol'] = ""
GBPUSD['fd'] = ""

j=0

for i in range(0,len(GBPUSD)):
    if j<len(results_gbpusd):
        if GBPUSD['Date'][i] >= date_list_calc[j]:
            GBPUSD['timestamp'][i] = results_gbpusd['inserttime'][j]
            GBPUSD['max'][i] = results_gbpusd['maxrate'][j]
            GBPUSD['min'][i] = results_gbpusd['minrate'][j]
            GBPUSD['mean'][i] = results_gbpusd['meanrate'][j]
            GBPUSD['vol'][i] = results_gbpusd['vol'][j]
            GBPUSD['fd'][i] = results_gbpusd['fd'][j]
        else:
            GBPUSD['timestamp'][i] = results_gbpusd['inserttime'][j]
            GBPUSD['max'][i] = results_gbpusd['maxrate'][j]
            GBPUSD['min'][i] = results_gbpusd['minrate'][j]
            GBPUSD['mean'][i] = results_gbpusd['meanrate'][j]
            GBPUSD['vol'][i] = results_gbpusd['vol'][j]
            GBPUSD['fd'][i] = results_gbpusd['fd'][j]
            j = j+1

In [ ]:
USDINR_final = USDINR.loc[USDINR['Week_tag'] == 1]
USDINR_final = USDINR_final.drop('Week_tag',axis =1 )

In [ ]:
EURUSD_final = EURUSD.loc[EURUSD['Week_tag'] == 1]
EURUSD_final = EURUSD_final.drop('Week_tag',axis =1 )
GBPUSD_final = GBPUSD.loc[GBPUSD['Week_tag'] == 1]
GBPUSD_final = GBPUSD_final.drop('Week_tag',axis =1 )

In [ ]:
USDINR_final['timestamp'].replace('', np.nan, inplace=True)
USDINR_final = USDINR_final.dropna()

In [ ]:
EURUSD_final['timestamp'].replace('', np.nan, inplace=True)
EURUSD_final = EURUSD_final.dropna()
GBPUSD_final['timestamp'].replace('', np.nan, inplace=True)
GBPUSD_final = GBPUSD_final.dropna()

In [290]:
USDINR_final

,Date,USDINR_fi,USDINR_ei,USDINR_ii,timestamp,max,min,mean,vol,fd
0,2022-07-01,0.775698,-0.754709,0.885303,2022-07-26 18:27:22,79.84425,79.84425,79.84425,0,0
5,2022-06-24,0.798670,-0.734775,0.895133,2022-07-26 18:27:22,79.84425,79.84425,79.84425,0,0
10,2022-06-17,0.800742,-0.652306,0.895478,2022-07-26 18:27:22,79.84425,79.84425,79.84425,0,0
15,2022-06-10,0.804241,-0.501222,0.891608,2022-07-26 18:27:22,79.84425,79.84425,79.84425,0,0
20,2022-06-03,0.804626,-0.315221,0.897860,2022-07-26 18:27:22,79.84425,79.84425,79.84425,0,0
...,...,...,...,...,...,...,...,...,...,...
3684,2007-12-19,-0.872069,-0.594929,-0.878139,2022-07-26 18:33:19,79.84125,79.84125,79.84125,0,0
3689,2007-12-11,-0.868906,-0.599422,-0.874126,2022-07-26 18:33:21,79.84125,79.84125,79.84125,0,0
3694,2007-12-04,-0.865518,-0.595341,-0.870107,2022-07-26 18:33:21,79.84125,79.84125,79.84125,0,0
3699,2007-11-27,-0.875813,-0.579956,-0.880020,2022-07-26 18:33:21,79.84125,79.84125,79.84125,0,0


In [291]:
EURUSD_final

,Date,EURUSD_fi,EURUSD_ei,EURUSD_ii,timestamp,max,min,mean,vol,fd
0,2022-07-01,-0.599368,0.799134,-0.756041,2022-07-26 18:14:46,1.013255,1.013245,1.01325,0.00001,129
5,2022-06-24,-0.640798,0.783042,-0.779387,2022-07-26 18:14:46,1.013255,1.013245,1.01325,0.00001,129
10,2022-06-17,-0.641248,0.731052,-0.781201,2022-07-26 18:14:46,1.013255,1.013245,1.01325,0.00001,129
15,2022-06-10,-0.639831,0.616655,-0.772226,2022-07-26 18:14:46,1.013255,1.013245,1.01325,0.00001,129
20,2022-06-03,-0.670963,0.479577,-0.815167,2022-07-26 18:14:46,1.013255,1.013245,1.01325,0.00001,129
...,...,...,...,...,...,...,...,...,...,...
3684,2007-12-19,0.947462,0.781994,0.946163,2022-07-26 18:21:03,1.012865,1.012865,1.012865,0.0,79
3689,2007-12-11,0.949668,0.806820,0.947864,2022-07-26 18:21:05,1.012875,1.012865,1.01287,0.00001,0
3694,2007-12-04,0.947937,0.793831,0.947287,2022-07-26 18:21:05,1.012875,1.012865,1.01287,0.00001,0
3699,2007-11-27,0.944516,0.743834,0.943744,2022-07-26 18:21:05,1.012875,1.012865,1.01287,0.00001,0


In [292]:
GBPUSD_final

,Date,GBPUSD_fi,GBPUSD_ei,GBPUSD_ii,timestamp,max,min,mean,vol,fd
0,2022-07-01,-0.653402,0.848099,-0.818215,2022-07-26 18:21:10,1.20382,1.20382,1.20382,0.0,0
5,2022-06-24,-0.671347,0.841039,-0.824644,2022-07-26 18:21:10,1.20382,1.20382,1.20382,0.0,0
10,2022-06-17,-0.662213,0.785992,-0.817523,2022-07-26 18:21:10,1.20382,1.20382,1.20382,0.0,0
15,2022-06-10,-0.654356,0.673212,-0.802645,2022-07-26 18:21:10,1.20382,1.20382,1.20382,0.0,0
20,2022-06-03,-0.664306,0.523662,-0.822347,2022-07-26 18:21:10,1.20382,1.20382,1.20382,0.0,0
...,...,...,...,...,...,...,...,...,...,...
3684,2007-12-19,0.734176,0.480485,0.741366,2022-07-26 18:27:15,1.203765,1.203735,1.20375,0.00003,111
3689,2007-12-11,0.738502,0.603236,0.738246,2022-07-26 18:27:17,1.203765,1.203745,1.203755,0.00002,146
3694,2007-12-04,0.737040,0.730463,0.730642,2022-07-26 18:27:17,1.203765,1.203745,1.203755,0.00002,146
3699,2007-11-27,0.720527,0.717221,0.713462,2022-07-26 18:27:17,1.203765,1.203745,1.203755,0.00002,146
